In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K

from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

#Get data 
cwd = os.path.normpath(os.getcwd())
df = pd.read_csv(cwd+'/data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

df_array = []
for idx in range(30):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

#df_array[3].head(3)

#Train, Validation and Test datasets
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]

dh = Datahandler()

X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = dh.min_max_scaling(train_df)
    val_df = dh.min_max_scaling(val_df)
    test_df = dh.min_max_scaling(test_df)

    # Sequencing
    train_sequences = dh.create_sequences(train_df, sequence_length)
    val_sequences = dh.create_sequences(val_df, sequence_length)
    test_sequences = dh.create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = dh.prepare_data(train_sequences, batch_size)
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = dh.prepare_data(val_sequences, batch_size)
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = dh.prepare_data(test_sequences, batch_size)

#General Hyperparameters
# #All models
horizon = 1
max_epochs = 100
m1 = ModelGenerator()
mh = Modelhandler()

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
#model_checkpoint = ModelCheckpoint('models/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
callbacks=[early_stopping, timing_callback, custom_callback] #model_checkpoint


In [2]:
def sum_weights(weight_list):
    """
    Return the sum of the listed weights. The is equivalent to avg of the weights
    """
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*weight_list):
        layer_mean = tf.math.reduce_mean(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

In [3]:
dense_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

# Dense Models

In [4]:
#Dense MODEL 1 ------------------------------------------------------------------
dense_architecture = "Dense_L3_U16"
dense_layers = 3
dense_units = 16

#Build and save global model
global_dense_moe = m1.build_dense_model(X_train[f'user{1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_dense_model(X_train[f'user{idx+1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )
        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_dense_model(X_train[f'user{idx+1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   
        
for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L3_U16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L3_U16/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L3_U16/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L3_U16/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L3_U16/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L3_U16/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L3_U16/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [5]:
dense_results.to_csv(f'evaluations/federated_learning/{dense_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,9.270104,0.357800,0.034682,0.000574,0.186227,0.001537,8.531152e+04,16999.935823,0.125190,0.002530
1,Dense_L3_U16,6.836421,0.363686,0.017359,0.000492,0.131745,0.001873,1.986780e+05,19862.416679,0.075392,0.001203
2,Dense_L3_U16,4.742015,0.377666,0.020776,0.000853,0.144117,0.002951,6.142955e+04,8695.047723,0.081255,0.003558
3,Dense_L3_U16,5.237831,0.375801,0.022885,0.001520,0.151222,0.005009,1.154816e+05,5626.637299,0.065652,0.001201
4,Dense_L3_U16,4.820683,0.383306,0.014566,0.000536,0.120678,0.002219,9.359337e+04,5165.220458,0.077718,0.001506
5,Dense_L3_U16,8.562604,0.348335,0.020261,0.000540,0.142332,0.001898,6.115138e+04,5364.869754,0.100222,0.000558
6,Dense_L3_U16,6.377668,0.364795,0.020593,0.000828,0.143483,0.002877,3.505160e+04,12526.682061,0.081484,0.000824
7,Dense_L3_U16,4.910159,0.371155,0.026614,0.000869,0.163122,0.002675,2.211706e+04,12157.387127,0.108089,0.001223
8,Dense_L3_U16,4.200814,0.374517,0.027021,0.000665,0.164373,0.002024,6.997551e+01,3.393702,0.093832,0.000709
9,Dense_L3_U16,6.942869,0.381752,0.015996,0.001264,0.126409,0.004945,2.072100e+06,229909.794817,0.071744,0.002811


In [6]:
#Dense MODEL 2 ------------------------------------------------------------------
dense_architecture = "Dense_L1_U4"
dense_layers = 1
dense_units = 4


#Build and save global model
global_dense_moe = m1.build_dense_model(X_train[f'user{1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_dense_model(X_train[f'user{idx+1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )
        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_dense_model(X_train[f'user{idx+1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   
        
for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L1_U4/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L1_U4/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L1_U4/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L1_U4/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L1_U4/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L1_U4/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L1_U4/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [7]:
dense_results.to_csv(f'evaluations/federated_learning/{dense_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,9.270104,0.357800,0.034682,0.000574,0.186227,0.001537,8.531152e+04,16999.935823,0.125190,0.002530
1,Dense_L3_U16,6.836421,0.363686,0.017359,0.000492,0.131745,0.001873,1.986780e+05,19862.416679,0.075392,0.001203
2,Dense_L3_U16,4.742015,0.377666,0.020776,0.000853,0.144117,0.002951,6.142955e+04,8695.047723,0.081255,0.003558
3,Dense_L3_U16,5.237831,0.375801,0.022885,0.001520,0.151222,0.005009,1.154816e+05,5626.637299,0.065652,0.001201
4,Dense_L3_U16,4.820683,0.383306,0.014566,0.000536,0.120678,0.002219,9.359337e+04,5165.220458,0.077718,0.001506
5,Dense_L3_U16,8.562604,0.348335,0.020261,0.000540,0.142332,0.001898,6.115138e+04,5364.869754,0.100222,0.000558
6,Dense_L3_U16,6.377668,0.364795,0.020593,0.000828,0.143483,0.002877,3.505160e+04,12526.682061,0.081484,0.000824
7,Dense_L3_U16,4.910159,0.371155,0.026614,0.000869,0.163122,0.002675,2.211706e+04,12157.387127,0.108089,0.001223
8,Dense_L3_U16,4.200814,0.374517,0.027021,0.000665,0.164373,0.002024,6.997551e+01,3.393702,0.093832,0.000709
9,Dense_L3_U16,6.942869,0.381752,0.015996,0.001264,0.126409,0.004945,2.072100e+06,229909.794817,0.071744,0.002811


In [8]:
#Dense MODEL 1 ------------------------------------------------------------------
dense_architecture = "Dense_L5_U32"
dense_layers = 5
dense_units = 32

#Build and save global model
global_dense_moe = m1.build_dense_model(X_train[f'user{1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_dense_model(X_train[f'user{idx+1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )
        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_dense_model(X_train[f'user{idx+1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   
        
for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L5_U32/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L5_U32/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L5_U32/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L5_U32/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L5_U32/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L5_U32/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_L5_U32/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [9]:
dense_results.to_csv(f'evaluations/federated_learning/{dense_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,9.270104,0.357800,0.034682,0.000574,0.186227,0.001537,8.531152e+04,16999.935823,0.125190,0.002530
1,Dense_L3_U16,6.836421,0.363686,0.017359,0.000492,0.131745,0.001873,1.986780e+05,19862.416679,0.075392,0.001203
2,Dense_L3_U16,4.742015,0.377666,0.020776,0.000853,0.144117,0.002951,6.142955e+04,8695.047723,0.081255,0.003558
3,Dense_L3_U16,5.237831,0.375801,0.022885,0.001520,0.151222,0.005009,1.154816e+05,5626.637299,0.065652,0.001201
4,Dense_L3_U16,4.820683,0.383306,0.014566,0.000536,0.120678,0.002219,9.359337e+04,5165.220458,0.077718,0.001506
...,...,...,...,...,...,...,...,...,...,...,...
85,Dense_L5_U32,6.283909,0.531796,0.026587,0.002682,0.162921,0.008119,1.404934e+05,31389.140080,0.103846,0.004705
86,Dense_L5_U32,6.549645,0.524428,0.020080,0.000929,0.141677,0.003296,1.765709e+05,19856.834635,0.095720,0.003438
87,Dense_L5_U32,7.583561,0.501474,0.011265,0.000067,0.106134,0.000314,5.725453e+04,2554.692247,0.046026,0.000310
88,Dense_L5_U32,7.434783,0.525422,0.016536,0.001347,0.128521,0.005238,4.284520e+05,26855.504914,0.092201,0.002417


# Bidirectional LSTM models

In [10]:
#Dense MODEL 1 ------------------------------------------------------------------
lstm_architecture = "L2_U8"
lstm_layers = 2
lstm_units = 8

#Build and save global model
global_dense_moe = m1.build_bilstm_model(X_train[f'user{1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
global_dense_moe.save(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_bilstm_model(X_train[f'user{idx+1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=lstm_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )
        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_bilstm_model(X_train[f'user{idx+1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=lstm_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   
        
for idx in range(len(df_array)):
    new_row = {
        'architecture': lstm_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U8/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U8/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U8/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U8/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U8/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U8/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U8/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [11]:
dense_results.to_csv(f'evaluations/federated_learning/{lstm_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,9.270104,0.357800,0.034682,0.000574,0.186227,0.001537,8.531152e+04,16999.935823,0.125190,0.002530
1,Dense_L3_U16,6.836421,0.363686,0.017359,0.000492,0.131745,0.001873,1.986780e+05,19862.416679,0.075392,0.001203
2,Dense_L3_U16,4.742015,0.377666,0.020776,0.000853,0.144117,0.002951,6.142955e+04,8695.047723,0.081255,0.003558
3,Dense_L3_U16,5.237831,0.375801,0.022885,0.001520,0.151222,0.005009,1.154816e+05,5626.637299,0.065652,0.001201
4,Dense_L3_U16,4.820683,0.383306,0.014566,0.000536,0.120678,0.002219,9.359337e+04,5165.220458,0.077718,0.001506
...,...,...,...,...,...,...,...,...,...,...,...
115,L2_U8,153.184486,5.665982,0.026752,0.001869,0.163493,0.005744,1.171177e+05,22248.795357,0.097763,0.003130
116,L2_U8,94.935386,5.493255,0.020107,0.001048,0.141766,0.003703,1.201851e+05,19734.861157,0.100160,0.004309
117,L2_U8,81.722582,5.396058,0.011001,0.000173,0.104882,0.000826,5.447468e+04,7895.359572,0.043588,0.000346
118,L2_U8,71.859792,5.113186,0.018028,0.001117,0.134226,0.004156,4.022427e+05,18167.809210,0.093665,0.002680


In [12]:
#Dense MODEL 1 ------------------------------------------------------------------
lstm_architecture = "L1_U8"
lstm_layers = 1
lstm_units = 8

#Build and save global model
global_dense_moe = m1.build_bilstm_model(X_train[f'user{1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
global_dense_moe.save(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_bilstm_model(X_train[f'user{idx+1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=lstm_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )
        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_bilstm_model(X_train[f'user{idx+1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=lstm_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   
        
for idx in range(len(df_array)):
    new_row = {
        'architecture': lstm_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L1_U8/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L1_U8/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L1_U8/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L1_U8/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L1_U8/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L1_U8/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L1_U8/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [13]:
dense_results.to_csv(f'evaluations/federated_learning/{lstm_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,9.270104,0.357800,0.034682,0.000574,0.186227,0.001537,8.531152e+04,16999.935823,0.125190,0.002530
1,Dense_L3_U16,6.836421,0.363686,0.017359,0.000492,0.131745,0.001873,1.986780e+05,19862.416679,0.075392,0.001203
2,Dense_L3_U16,4.742015,0.377666,0.020776,0.000853,0.144117,0.002951,6.142955e+04,8695.047723,0.081255,0.003558
3,Dense_L3_U16,5.237831,0.375801,0.022885,0.001520,0.151222,0.005009,1.154816e+05,5626.637299,0.065652,0.001201
4,Dense_L3_U16,4.820683,0.383306,0.014566,0.000536,0.120678,0.002219,9.359337e+04,5165.220458,0.077718,0.001506
...,...,...,...,...,...,...,...,...,...,...,...
145,L1_U8,33.676727,2.121505,0.024427,0.002924,0.156108,0.009309,9.321534e+04,15847.449051,0.090478,0.007047
146,L1_U8,32.114441,2.139032,0.019530,0.000536,0.139740,0.001910,1.488246e+05,12632.174632,0.096971,0.002347
147,L1_U8,31.212766,2.119599,0.010716,0.000109,0.103517,0.000528,6.915486e+04,4098.321021,0.049376,0.001868
148,L1_U8,35.602278,2.127649,0.017291,0.000382,0.131491,0.001449,3.971448e+05,130127.463730,0.091724,0.002913


In [14]:
#Dense MODEL 1 ------------------------------------------------------------------
lstm_architecture = "L2_U20"
lstm_layers = 2
lstm_units = 20

#Build and save global model
global_dense_moe = m1.build_bilstm_model(X_train[f'user{1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
global_dense_moe.save(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_bilstm_model(X_train[f'user{idx+1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=lstm_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )
        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/LSTM/global_lstm_model/{lstm_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_bilstm_model(X_train[f'user{idx+1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=lstm_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   
        
for idx in range(len(df_array)):
    new_row = {
        'architecture': lstm_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U20/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U20/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U20/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U20/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U20/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U20/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/LSTM/global_lstm_model/L2_U20/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [15]:
dense_results.to_csv(f'evaluations/federated_learning/{lstm_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,9.270104,0.357800,0.034682,0.000574,0.186227,0.001537,8.531152e+04,16999.935823,0.125190,0.002530
1,Dense_L3_U16,6.836421,0.363686,0.017359,0.000492,0.131745,0.001873,1.986780e+05,19862.416679,0.075392,0.001203
2,Dense_L3_U16,4.742015,0.377666,0.020776,0.000853,0.144117,0.002951,6.142955e+04,8695.047723,0.081255,0.003558
3,Dense_L3_U16,5.237831,0.375801,0.022885,0.001520,0.151222,0.005009,1.154816e+05,5626.637299,0.065652,0.001201
4,Dense_L3_U16,4.820683,0.383306,0.014566,0.000536,0.120678,0.002219,9.359337e+04,5165.220458,0.077718,0.001506
...,...,...,...,...,...,...,...,...,...,...,...
175,L2_U20,131.717002,7.075551,0.026497,0.002363,0.162669,0.007338,8.336910e+04,12000.769064,0.098456,0.007803
176,L2_U20,100.295987,7.269560,0.019999,0.000368,0.141414,0.001306,1.195685e+05,6071.011839,0.096010,0.002549
177,L2_U20,86.900287,7.180213,0.010985,0.000143,0.104806,0.000683,5.179564e+04,6194.489330,0.043027,0.001408
178,L2_U20,155.755443,7.004049,0.017968,0.000646,0.134032,0.002403,4.636550e+05,16749.583148,0.092455,0.001296
